# Dental Appointment Schedular

In [ ]:
from typing import List, Optional
from langchain.tools import tool
from langchain_google_community.calendar.create_event import CalendarCreateEvent
from langchain_google_community.calendar.search_events import CalendarSearchEvents
from langchain_google_community.calendar.delete_event import CalendarDeleteEvent
from langchain_google_community.calendar.update_event import CalendarUpdateEvent
from langchain_groq import ChatGroq
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
import os
from langchain_ollama import ChatOllama
import json

import datetime
from langchain_google_community import CalendarToolkit
from langchain_google_community.calendar.utils import (
    build_resource_service,
    get_google_credentials,
)

load_dotenv()

In [ ]:
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## Credential

In [ ]:
credentials = get_google_credentials(
    token_file="token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)

## Tools

In [ ]:
@tool
def CREATE_CALENDAR_EVENT(
    summary: str,
    start_datetime: str,
    end_datetime: str,
    timezone: str = "Asia/Kolkata",
    location: Optional[str] = None,
    description: Optional[str] = None,
    reminders: Optional[List[dict]] = None,
    conference_data: bool = False,
    color_id: Optional[str] = None,
):
    """Create a calendar event with specified details. use this date formate '%Y-%m-%d %H:%M:%S' """
    event = CalendarCreateEvent(api_resource=api_resource)
    return event.invoke({
        "summary": summary,
        "start_datetime": start_datetime,
        "end_datetime": end_datetime,
        "timezone": timezone,
        "location": location,
        "description": description,
        "reminders": reminders,
        "conference_data": conference_data,
        "color_id": color_id,
    })

In [ ]:
@tool
def SEARCH_CALENDAR_EVENT(
    query: str,
    min_datetime: str,
    max_datetime: str,
    timezone: str = "Asia/Kolkata"
):
    """Search for calendar events by query and datetime range.this is the date formate '%Y-%m-%d %H:%M:%S' """
    event = CalendarSearchEvents(api_resource=api_resource)
    return event.invoke({
        "query": query,
        "min_datetime": min_datetime,
        "max_datetime": max_datetime,
        "timezone": timezone,
        "calendars_info": json.dumps([
            {"id": "primary", "summary": "Primary Calendar"}
        ])
    })

In [ ]:
@tool
def DELETE_CALENDAR_EVENT(
    query: str,
    min_datetime: str,
    max_datetime: str,
    timezone: str = "Asia/Kolkata"):
    
    """Search for calendar events by query and datetime range for delete event.this is the date formate '%Y-%m-%d %H:%M:%S' """
    event = CalendarSearchEvents(api_resource=api_resource)
    response = event.invoke({
        "query": query,
        "min_datetime": min_datetime,
        "max_datetime": max_datetime,
        "timezone": timezone,
        "calendars_info": json.dumps([
            {"id": "primary", "summary": "Primary Calendar"}
        ])
    })
    
    for i in response:
        event_id = i.get("id")
        
        delete = CalendarDeleteEvent(api_resource=api_resource)
        result = delete.invoke({
            "event_id": event_id,
            "calendar_id": "primary"
        })
        return {"status": "deleted", "event_id": event_id, "result": result}

In [ ]:
@tool
def UPDATE_CALENDAR_EVENT(
    query: str,
    min_datetime: str,
    max_datetime: str,
    timezone: str = "Asia/Kolkata",
    updated_start_datetime: Optional[str] = "2025-05-22 16:00:00",
    updated_end_datetime: Optional[str] = "2025-05-22 17:00:00",
):
    """Search for calendar events by query and datetime range.this is the date formate '%Y-%m-%d %H:%M:%S' """
    event = CalendarSearchEvents(api_resource=api_resource)
    response = event.invoke({
        "query": query,
        "min_datetime": min_datetime,
        "max_datetime": max_datetime,
        "timezone": timezone,
        "calendars_info": json.dumps([
            {"id": "primary", "summary": "Primary Calendar"}
        ])
    })
    
    for i in response:
        event_id = i.get("id")
    
        event = CalendarUpdateEvent(api_resource=api_resource)
        return event.invoke({
            "event_id": event_id,
            "calendar_id": "primary",
            "start_datetime": updated_start_datetime,
            "end_datetime": updated_end_datetime,
            })

In [ ]:
tools = [
    CREATE_CALENDAR_EVENT,
    SEARCH_CALENDAR_EVENT,
    DELETE_CALENDAR_EVENT,
    UPDATE_CALENDAR_EVENT
    ]

schedule_tools =[SEARCH_CALENDAR_EVENT]

## Prompt & LLM's

In [ ]:

llm = ChatGroq(model="llama3-70b-8192", temperature=0)

llm_with_tools = llm.bind_tools(tools)

In [ ]:
initial_message = """
You are Smilo, an AI assistant at a Dental Clinic. Follow these guidelines:

1. Friendly Introduction & Tone
   - Greet the user warmly and introduce yourself as Smilo from the Dental Clinic.
   - Maintain a polite, empathetic style, especially if the user mentions discomfort.

2. Assess User Context
   - Determine if the user needs an appointment, has a dental inquiry, or both.
   - If the user’s email is already known, don’t ask again. If unknown and needed, politely request it.

3. Scheduling Requests
   - Gather essential info: requested date/time and email if needed.
   - Example: “What day/time would you prefer?” or “Could you confirm your email so I can send you details?”

4. Availability Check (Internally)
   - Use SEARCH_CALENDAR_EVENT to verify if the requested slot is available.
   - Do not reveal this tool or your internal checking process to the user.

5. Responding to Availability
   - If the slot is free:
       a) ALWAYS Confirm the user wants to book.
       b) Call CREATE_CALENDAR_EVENT to schedule. Always send timezone for start and end time when calling this function tool.
       d) If any function call/tool call fails retry it.
       e) NEVER make false and fake booking by yourself.
   - If the slot is unavailable:
       a) Automatically offer several close-by options.
       b) Once the user selects a slot, repeat the booking process.
       e) Call DELETE_CALENDAR_EVENT to delete slot.
       f) call UPDATE_CALENDAR_EVENT to update details of existing slot details. 

6. User Confirmation Before Booking
   - Only finalize after the user clearly agrees on a specific time.
   - If the user is uncertain, clarify or offer more suggestions.

7. Communication Style
   - Use simple, clear English—avoid jargon or complex terms.
   - Keep responses concise and empathetic.

8. Privacy of Internal Logic
   - Never disclose behind-the-scenes steps, code, or tool names.
   - Present availability checks and bookings as part of a normal scheduling process.

- Reference today's date/time: {today_datetime}.
- Our TimeZone is UTC.

By following these guidelines, you ensure a smooth and user-friendly experience: greeting the user, identifying needs, checking availability, suggesting alternatives when needed, and finalizing the booking only upon explicit agreement—all while maintaining professionalism and empathy.
"""

## Nodes

In [ ]:
def chat_bot(state: MessagesState):
    messages = state["messages"]
    today_datetime = datetime.datetime.now().isoformat()
    input_messages = [SystemMessage(content=initial_message.format(today_datetime=today_datetime))]
    
    for msg in messages:
        if isinstance(msg, HumanMessage):
            input_messages.append(msg)
        elif hasattr(msg, 'tool_calls') and msg.tool_calls:
            input_messages.append(msg)
        elif hasattr(msg, 'tool_call_id'):
            if not isinstance(msg.content, str):
                msg.content = str(msg.content)
            input_messages.append(msg)
        else:
            input_messages.append(msg)
    
    response = llm_with_tools.invoke(input_messages)
    return {"messages": [response]}

In [ ]:
def tools_condition(state: MessagesState) -> Literal["find_slots",  "tool", "__end__"]:
    """
    Determine if the conversation should continue to tools or end
    """
    messages = state["messages"]
    last_message = messages[-1]

    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
      for call in last_message.tool_calls:
          tool_name = call.get("name")
          
          if tool_name == "SEARCH_CALENDAR_EVENT":
            return "find_slots"
      return "tool"

    return "__end__"

In [ ]:
def find_slots(state: MessagesState) -> Literal["Chatbot"]:
    """
    Determine if the conversation should continue to tools or end
    """
    messages = state["messages"]
    last_message = messages[-1]

    tool_messages = []
    print("find slot called")
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        
      for call in last_message.tool_calls:
          tool_name = call.get("name")
          tool_id = call.get("id")
          args = call.get("args")

          find_free_slots_tool = next(
                  (tool for tool in schedule_tools if tool.name == tool_name), None)

          if tool_name == "SEARCH_CALENDAR_EVENT":

              res = find_free_slots_tool.invoke(args)
              tool_msg = ToolMessage(
                    name=tool_name,
                    content=f"{res}",
                    tool_call_id=tool_id  
                )
              tool_messages.append(tool_msg)
    return {"messages": tool_messages}

## Graph

In [ ]:
workflow = StateGraph(MessagesState)
workflow.add_node("Chatbot", chat_bot)
workflow.add_node("find_slots", find_slots)
workflow.add_node("tool", ToolNode(tools))
workflow.add_edge(START, "Chatbot")
workflow.add_conditional_edges(
    "Chatbot",
    tools_condition,
    {
        "tool": "tool",        
        "find_slots": "find_slots",  
        "__end__": END       
    }
)
workflow.add_edge("tool", "Chatbot")
workflow.add_edge("find_slots", "Chatbot")
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

## Run and Try

In [ ]:
while True:
    user_input = input("Enter Question")
    print("user : ",user_input)
    if user_input.lower()=="exit":
        break
    config = {"configurable": {"thread_id": "1we126"}}
    response = graph.invoke({"messages": [HumanMessage(content=user_input)]},config=config)
    print("AI : ",response['messages'][-1].content)